# Using pretrained models (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## Основные понятия

In [6]:
# Использования токена:
import os
from huggingface_hub import login
login(os.getenv("HF_TOKEN"))

In [7]:
from transformers import pipeline

camembert_fill_mask = pipeline("fill-mask", model="camembert-base")
results = camembert_fill_mask("Le camembert est <mask> :)")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [ ]:
results

In [ ]:
for word in results:
  print(word['token_str'],": ", word['sequence'])

____________________________
____________________________

Вы также можете создать экземпляр контрольной точки, используя архитектуру модели напрямую:

In [ ]:
from transformers import CamembertTokenizer, CamembertForMaskedLM

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForMaskedLM.from_pretrained("camembert-base")

Однако мы рекомендуем использовать вместо этого  *__Auto...классы__*, поскольку они по своей сути не зависят от архитектуры. В то время как предыдущий пример кода ограничивает пользователей контрольными точками, загружаемыми в архитектуре CamemBERT, использование классов Auto* упрощает переключение контрольных точек:

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModelForMaskedLM.from_pretrained("camembert-base")

_______________________________________________________________
_______________________________________________________________

## Импровизация

In [11]:
# from transformers import CamembertModel, CamembertTokenizer
# # You can replace "camembert-base" with any other model from the table, e.g. "camembert/camembert-large".
# tokenizer = CamembertTokenizer.from_pretrained("camembert/camembert-base-wikipedia-4gb")
# camembert = CamembertModel.from_pretrained("camembert/camembert-base-wikipedia-4gb")
# camembert.eval()  # disable dropout (or leave in train mode to finetune)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("camembert/camembert-base-wikipedia-4gb")

config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

In [14]:
sentence = "J'aime le camembert !"
tokenized_sentence = tokenizer.tokenize(sentence)
tokenized_sentence

['▁J', "'", 'aime', '▁le', '▁cam', 'ember', 't', '▁!']

In [34]:
encoded_sentence = tokenizer(sentence, return_tensors='pt')
encoded_sentence

{'input_ids': tensor([[    5,   221,    10, 10600,    14,  8952, 10540,    75,  1114,     6]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [52]:
encoded_sentence.input_ids

tensor([[    5,   221,    10, 10600,    14,  8952, 10540,    75,  1114,     6]])

In [53]:
import torch
from transformers import AutoModel
model_camembert = AutoModel.from_pretrained("camembert/camembert-base-wikipedia-4gb")
# AutoModelForMaskedLM — используется для предсказания скрытых слов, а не получения эмбеддингов.
# AutoModel — используется, когда нужны векторные представления (эмбеддинги).

# Получаем эмбеддинги
with torch.no_grad():  # Отключаем градиенты (ускоряет вычисления)
    outputs = model_camembert(**encoded_sentence)

Some weights of CamembertModel were not initialized from the model checkpoint at camembert/camembert-base-wikipedia-4gb and are newly initialized: ['roberta.embeddings.word_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Здесь мы получаем эмбеддинги из модели CamemBERT:
`model_camembert(encoded_sentence)` — прогоняем тензор через предобученную модель CamemBERT.  
Выход модели — два значения:  
- `embeddings` -  эмбеддинги (векторы признаков) для каждого токена,
- Второй выход "`_`" — обычно содержит `hidden states` или `attention weights`, но он здесь не используется.

In [57]:
# Выводим размерность эмбеддингов
print(outputs.last_hidden_state.shape)  # torch.Size([1, sequence_length, 768])

torch.Size([1, 10, 768])


## Использование эмбедингов для классификации

In [ ]:
pip install transformers torch

In [59]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer

# Загружаем токенизатор и предобученную модель CamemBERT
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
camembert = AutoModel.from_pretrained("camembert-base")

# Определяем классификационную модель
class CamembertClassifier(nn.Module):                   # class CamembertClassifier(nn.Module) - создаём новую модель CamembertClassifier,
                                                        # которая наследуется от `torch.nn.Module`. Это стандартный способ объявлять модели в PyTorch.
    def __init__(self, camembert_model, num_classes=2):
        super(CamembertClassifier, self).__init__()
        self.camembert = camembert_model                # Загружаем предобученный CamemBERT из transformers.
                                                        # Эта модель уже предобучена на французском языке и умеет создавать эмбеддинги для текста.
        self.dropout = nn.Dropout(0.3)                  # Dropout для регуляризации. Dropout(0.3) случайно зануляет 30% нейронов во время обучения.
                                                        # Это предотвращает переобучение модели.
        self.classifier = nn.Linear(768, num_classes)   # Линейный слой (768 -> num_classes). nn.Linear(768, num_classes) — обычный полносвязный слой:
                                                        # Вход: вектор размером 768 (эмбеддинг [CLS]).
                                                        # Выход: num_classes (например, 2, если классифицируем позитив/негатив).
                                                        # Этот слой предсказывает логиты (числа перед softmax).

    def forward(self, input_ids, attention_mask):       # Этот метод определяет, как данные проходят через модель.
        with torch.no_grad():                           # CamemBERT замораживаем (можно разморозить для дообучения)
            outputs = self.camembert(input_ids=input_ids, attention_mask=attention_mask)   # Пропускаем текст через CamemBERT,
                                                                                          # self.camembert(...) получает эмбеддинги токенов.
                                                                                          # torch.no_grad() замораживает веса CamemBERT,
                                                                                          #чтобы он не обучался (можно убрать, если хотим дообучать).
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # Берём эмбеддинг [CLS]. utputs.last_hidden_state имеет размер [batch_size, seq_len, 768].
                                                            # [CLS] токен (первый токен в предложении) содержит сводную информацию о всём тексте.
                                                            # [:, 0, :] → Берём первый токен (размер 768).

        x = self.dropout(cls_embedding)                     # Пропускаем через Dropout (регуляризация)
        logits = self.classifier(x)                         # Пропускаем через линейный слой. self.classifier(x) даёт логиты (числа перед softmax).
                                                            # Если у нас 2 класса, выход будет [batch_size, 2] (значения для каждого класса).
        return logits                                       #  # Выход: [batch_size, num_classes]

# Создаём модель
num_classes = 2  # Например, два класса: позитивный/негативный
model = CamembertClassifier(camembert, num_classes)


<small>Кстати. Почему forward не вызывается явно?  
В PyTorch, если мы пишем: `output = model(x)`, PyTorch автоматически вызывает:
`output = model.forward(x)` То есть, в PyTorch метод forward не нужно вызывать вручную.  
В CamemBERT-классификаторе Когда мы пишем:  
`logits = model(input_ids, attention_mask)` PyTorch автоматически вызывает:
`logits = model.forward(input_ids, attention_mask)`. Поэтому forward не вызывается вручную, но он всегда работает в момент вызова модели.</small>

Без обучения модель не сможет правильно классифицировать предложения, потому что:__CamemBERT – это языковая модель, а не классификатор__ (Предобученная модель CamemBERT не предназначена для классификации "из коробки". Она обучена понимать французский язык и предсказывать пропущенные слова (Masked Language Model). Для классификации нам нужно дообучить линейный слой на размеченных данных.)

Она __умеет получать эмбеддинги__, но __не обучен распознавать "положительный" или "отрицательный" тон__. Она просто выдаст случайные логиты, так как __классификационный слой (линейный nn.Linear)__ изначально не обучен. Линейный слой изначально имеет случайные веса.  
Когда мы создаём self.classifier = nn.Linear(768, num_classes), его веса инициализируются случайными значениями.  

Поэтому, чтобы модель правильно классифицировала, нам нужно её обучить. Вот минимальный код обучения:

In [64]:
from torch.optim import AdamW  # Используем PyTorch версию

# 1️⃣ Создаём обучающие данные (простые примеры)
texts = ["J'adore ce fromage, il est délicieux.",  # Позитивный (1)
         "Je déteste ce fromage, il est horrible."]  # Негативный (0)
labels = torch.tensor([1, 0])  # Метки классов

# 2️⃣ Токенизируем данные
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# 3️⃣ Оптимизатор и функция потерь
optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

# 4️⃣ Запускаем обучение
model.train()
num_epochs = 10
for epoch in range(num_epochs):
    optimizer.zero_grad()

    logits = model(encodings["input_ids"], encodings["attention_mask"])
    loss = criterion(logits, labels)

    loss.backward()
    optimizer.step()

    print(f"Эпоха {epoch + 1}/{num_epochs}, Потери: {loss.item():.4f}")

# 5️⃣ Проверяем после обучения
model.eval()
test_text = "Ce fromage est affreux."
test_encoding = tokenizer(test_text, return_tensors="pt")
logits = model(test_encoding["input_ids"], test_encoding["attention_mask"])
prediction = torch.argmax(logits, dim=1).item()
print("Предсказанный класс:", "Позитивный" if prediction == 1 else "Негативный")


Эпоха 1/10, Потери: 0.6305
Эпоха 2/10, Потери: 0.6806
Эпоха 3/10, Потери: 0.6908
Эпоха 4/10, Потери: 0.6594
Эпоха 5/10, Потери: 0.6700
Эпоха 6/10, Потери: 0.7108
Эпоха 7/10, Потери: 0.6986
Эпоха 8/10, Потери: 0.6862
Эпоха 9/10, Потери: 0.6603
Эпоха 10/10, Потери: 0.6955
Предсказанный класс: Позитивный


Еще пример, но, как видим, обучение модели на одном примере - недостаточно, так как ошибка [1, 1], вместо [1, 0]

In [65]:
texts_2 = ["J'adore ce fromage, il est délicieux.", "Je déteste ce fromage, il est horrible."]

1. __Токенизируем текст:__  

In [66]:
encodings_2 = tokenizer(texts_2, padding=True, truncation=True, return_tensors="pt")

2. __Пропускаем через модель:__  

In [67]:
logits_2 = model(encodings_2["input_ids"], encodings_2["attention_mask"])

3. __Предсказываем класс:__  

In [68]:
predictions = torch.argmax(logits_2, dim=1)
print(predictions.tolist())  # Ожидаемый результат: [1, 0]

[1, 1]


4. __Итог__  
🔹 CamemBERT превращает текст в эмбеддинги.  
🔹 Берём эмбеддинг [CLS] как представление всего текста.  
🔹 Линейный слой предсказывает класс (позитив / негатив).  
🔹 Dropout помогает избежать переобучения.  
🔹 Модель можно обучать, используя AdamW и CrossEntropyLoss.  